# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [2070]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2071]:

def dna_to_rna(s):
    transcribe = {"T": "U"}
    rna = [transcribe[nucl] if nucl in transcribe else nucl for nucl in s]
    return ''.join(rna)

    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

"""There is only one letter that should be substituted, there for we only need to search for that one. So we iterate over the given string, and the comprehensive list, is checking each character in the string to see if it is in the dictionary. If it's not the character remains the same, if it matches the value is return "U". The comprehansive list returns a list, where each charater has it own index. There for we need to use the join method for the list to be returned as a string """ 

### Discussion

"""It worked as excepted, honestly there is not much to discuss yet"""

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [2072]:
import requests
import re
from bs4 import BeautifulSoup

def get_data():
    # URL of the HTML file to scrape
    url = "https://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Extract the HTML content from the response
        html_content = response.text
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the <pre> tag containing the codon table
        pre_tag = soup.find('pre')

        # Extract the text from the <pre> tag
        codon_table_text = pre_tag.get_text()
        
        # Define the regular expression pattern to search for column names
        pattern = r'fields:\s*\[triplet\]\s*\[amino acid\]\s*\[fraction\]\s*\[frequency: <strong>per thousand</strong>\]\s*\(\[number\]\)'
        
        # Search for the pattern in the HTML content
        match = re.search(pattern, html_content, re.IGNORECASE)
        
        
        # Access the matched string instead of object
        column_names = match.group()
        
        # Regular expression pattern to remove most extra charaters
        pattern = r"\[(.*?)\]"

        # Extracting the column names with out the charaters from above
        matches = re.findall(pattern, column_names)

        # Formatting the extracted column names to a list again
        formatted_string = " ".join(matches)
        
        # Remove HTML tags and format column names
        formatted_string = re.sub(r'<.*?>', '', formatted_string)
        
        # Split the formatted string into a list of column names
        formatted_list = formatted_string.split()
        
        # Create a substring for the frequency column to get it in the right format
        new_sub = ' '.join([formatted_list[5], formatted_list[6]])
        new_sub = ''. join([formatted_list[4][:-1], f"({new_sub}):"])
        
        # Combine the formatted column names to the final form
        formatted_column = formatted_list[:4] + [new_sub] + formatted_list[-1:]

    else:
        # Print error message if request fails
        print("Failed to retrieve HTML content. Status code:", response.status_code)

    # Return the formatted column names and codon table text
    return formatted_column, codon_table_text

    
def get_dict():
    # Get formatted column names and codon table text
    column_names, codon_table = get_data()
    
    # Split the codon table text into rows
    codon_rows = codon_table.split('\n')
    
    # Remove whitespace
    codon_rows = [row.strip() for row in codon_rows if row.strip()]

    # Initialize an empty dictionary to store codon-amino acid pairs
    codon_dict = {}
    
    # Extract the codon sequences and amino acids from each row
    for row in codon_rows:
        # Split the row into individual elements
        elements = row.split()

        # The values with a extra whitespace fx ( 63237) will create and extra value '(' in the list when doing.split()
        # so we remove that extra ",", because we dont use it any way
        elements = [x for x in elements if x != '(']
    
        # Select the right elements for codon and amino acid
        codon_sequence = elements[0]
        amino_acid = elements[1]
        codon_dict[codon_sequence] = amino_acid
        
        codon_sequence = elements[5]
        amino_acid = elements[6]
        codon_dict[codon_sequence] = amino_acid
        
        codon_sequence = elements[10]
        amino_acid = elements[11]
        codon_dict[codon_sequence] = amino_acid
        
        codon_sequence = elements[15]
        amino_acid = elements[16]
        codon_dict[codon_sequence] = amino_acid
    
    # Return the dictionary mapping codons to amino acids
    return codon_dict

    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)
    

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

"""The idea is to get the wanted data from the webpage, by webscraping. The provided webpage was a copy, by entering the orginal webpage it made webscraping easier. After the wanted data is collected, it is being put on the right format, mostly the column names. Hower ever the column names are not being used in the get_dict function, and is more for purpos of understanding the columns. 

we manipulate the table from get_data function in get_dict. Here we split it into rows, and remove whitespace. Now we split each row into serpate values, and we append the wanted values to the dictionary. 
"""

### Discussion

""The solution works perfect, but the part with formatting and extracting the column names, are not strictly nessary in this solution. How ever for a dataset, with more unstructored data, the solution we accesing the wanted values based on their indicies may cause problems.

The length of the dicionary is 64, wich also is the correct amount of codons""

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [2073]:

def get_dict_list():
    previous_dict = get_dict()
    # New dictionary
    amino_acid_dict = {}
    for key, value in previous_dict.items():
        if not value in amino_acid_dict:
            amino_acid_dict[value] = [key]
        else:
            amino_acid_dict[value].append(key)
    
    
    
    return amino_acid_dict

if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

"""Using the result from the prevoius function, since it contain all the data we need, so we only need to orginize it differently. We creacte a new dictionart, and loop over both values and keys, and if the values not are in the dictionary they are added as a list. If that is false it means that the value is already there, and then we just need to append the new value to the new value list in the dictionary"""

### Discussion

"""The solution works well, when it is given a dictionary where codons are  keys and amino acid are values. By using .items() we can acess both at the same time in a loop. 

by typing return len(amino_acid_dict). we get 21, that is because * altso count, otherwise we have 20 different amino acid"""

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [2074]:
def rna_to_prot(s):
    # Get the dictionary mapping RNA sequences to amino acids
    rna_to_aa = get_dict()

    # Split the RNA sequence into codons of length 3
    codons = [s[i:i+3] for i in range(0, len(s), 3)]

    # Convert each codon into its corresponding amino acid
    protein_sequence = ""
    for codon in codons:
        # Check if the codon is in the dictionary
        if codon in rna_to_aa:
            amino_acid = rna_to_aa[codon]
            protein_sequence += amino_acid
        else:
            # If the codon is not found, consider it as a stop codon
            protein_sequence += "*"

    return protein_sequence

def dna_to_prot(s):
    # Convert DNA sequence to RNA sequence
    rna_sequence = dna_to_rna(s)
    # Convert RNA sequence to protein sequence
    protein_sequence = rna_to_prot(rna_sequence)
    return protein_sequence

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

"""get_dict_list() retrieves a dictionary that maps RNA codons to their corresponding amino acids.
It utilizes the get_dict() function from Exercise 2 to obtain the initial dictionary.
It then creates a new dictionary where the keys are amino acids and the values are lists of RNA codons that code for those amino acids.

rna_to_prot(s) splits the RNA sequence into codons and looks up each codon in the conversion dictionary obtained from get_dict_list().
For each codon, it retrieves the corresponding amino acid and appends it to the protein sequence.
If we cant find the codon in the dictionary a * is append to the protein squence

The last function is calling the functions for the final result

### Discussion

The code seems to forfill the descired purpose. However it dont handle typing errors, but that was not given as a task. 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [2075]:

def get_probabability_dict():

    # Get formatted column names and codon table text
    column_names, codon_table = get_data()
    
    # Split the codon table text into rows
    codon_rows = codon_table.split('\n')
    # Remove any empty rows
    codon_rows = [row.strip() for row in codon_rows if row.strip()]
    
    # Initialize an empty dictionary to store codon adaptation probabilities
    probability_dict = {}
    
    # Iterate over each row in the codon table
    for row in codon_rows:
        # Split the row into individual elements
        elements = row.split()
        elements = [x for x in elements if x != '(']
        # Iterate over the elements with a step size of 5
        for i in range(1, len(elements), 5):
            codon_sequence = elements[i - 1]
        
            # handle that some values have ()
            if "(" in elements[i + 3]:
                number_value = elements[i + 3][1:-1]
            else:
                number_value = elements[i + 3][:-1]
            
            # Convert the number value to float and divide by 1000 to get the probability
            probability = float(elements[i + 2]) / (float(number_value) / 1000)
            
    
            probability_dict[codon_sequence] = probability
            

    return probability_dict
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    print(codon_to_prob)
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
            ))

{'UUU': 0.024639576199289373, 'UCU': 0.024567205044035097, 'UAU': 0.0246117099288076, 'UGU': 0.02463334657956687, 'UUC': 0.024615250299505755, 'UCC': 0.024621222659314608, 'UAC': 0.02458198574244827, 'UGC': 0.024560062998510802, 'UUA': 0.02468890378060863, 'UCA': 0.024574577800696145, 'UAA': 0.02482313516197096, 'UGA': 0.025301643025443965, 'UUG': 0.02453927044178296, 'UCG': 0.024523601179362274, 'UAG': 0.024915132828801895, 'UGG': 0.0246454877286009, 'CUU': 0.024603226377640886, 'CCU': 0.024536161394663455, 'CAU': 0.02467676829420141, 'CGU': 0.024375842997903675, 'CUC': 0.02460339577072648, 'CCC': 0.024607889438492705, 'CAC': 0.02460433459939744, 'CGC': 0.02455633317277269, 'CUA': 0.02476345739137613, 'CCA': 0.024562596833314437, 'CAA': 0.024506336780823694, 'CGA': 0.02472483649704897, 'CUG': 0.024568789819586913, 'CCG': 0.02450545157509678, 'CAG': 0.024569442079695514, 'CGG': 0.024543311409410424, 'AUU': 0.024597485214605374, 'ACU': 0.024549810816534203, 'AAU': 0.024648362116337367, 

### Idea of solution

Im unsure excaly how the possiblities is excepted to be calculated to, so i did the way i thought [frequency: per thousand]/([number]/1000). Again im a bit unsure exacly how you expect the output to be, an example would help. 

The Idea is to call the get_data function and use the data here from the values we want comes at a step of 5. with in the for loop we need to be aware of ( and () in the number values. When we have the values we want, we calculate the possibility using the formula above. 

### Discussion

again we have the len of 64, and the values seems to be small, and in a realistic interval for possibilities. So it seems to work

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [2076]:
class ProteinToMaxRNA:
    
    def __init__(self):
        # Initialize the necessary data structures
        self.codon_to_prob = get_probabability_dict()
        self.amino_acid_to_codons = get_dict_list()

    def convert(self, s):
        max_rna_sequence = ""
        for amino_acid in s:
            # Get the dictionary of codons for the current amino acid
            codons_dict = self.amino_acid_to_codons.get(amino_acid)
            if codons_dict:
                # Find the codon with the highest probability
                max_codon = max(codons_dict, key=lambda x: self.codon_to_prob.get(x, 0))
                max_rna_sequence += max_codon
            else:
                # If the amino acid is not found, add a placeholder
                max_rna_sequence += "???"
        return max_rna_sequence


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUAACGCCCAUACAGAAUCGAGCG


### Idea of solution

"""we Initialize the functions we need and there by getting the dictionary from them. We Loop over the RNA  and match it with the codon with the highest probability"""

### Discussion

The string "LTPIQNRA" is converted to CUAACGCCCAUACAGAAUCGAGCG based on the prevouis functions

### Discussion

For the amino acid "L", the most probable codon is "CUA" .
For the amino acid "T", the most probable codon is "ACC".
Etc













 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 













 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 





AACGUGAUUUC
CUAACGCCCAUACAGAAUCGAGCG
CUGACCCCCAUCCAGAACAGAGCC 

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [2126]:
import random

def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    # Ensure the probabilities sum up to 1.0
    total_prob = sum(dist.values())
    if total_prob != 1.0:
        raise ValueError("Probabilities must sum up to 1.0")
    
    # Generate a random number between 0 and 1
    rand_num = random.random()
    
    # Iterate through the events and their probabilities
    cumulative_prob = 0
    for event, prob in dist.items():
        cumulative_prob += prob
        if rand_num <= cumulative_prob:
            return event

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))


C, T, C, C, T, C, C, G, G, A, C, T, C, T, T, T, C, C, T, T, T, C, T, T, C, T, C, G, T


### Idea of solution

returns:
C, C, C, C, T, T, C, A, C, G, C, T, T, C, C, C, G, A, T, T, C, T, G, T, T, T, C, T, T

our function takes a probability distribution as a dictionary from events to their probabilities, and it uses random.uniform to produce values uniformly at random from the range $[0,1)$. The function then partitions the unit interval according to cumulative probabilities and selects the event accordingly.

### Discussion

we could also have used numpys np.choice method 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [2160]:
import random

class ProteinToRandomRNA:
    def __init__(self, codon_probs, aa_to_rna):
        self.codon_probs = codon_probs
        self.aa_to_rna = aa_to_rna

    def convert(self, protein_sequence):
        rna_sequence = ""
        for amino_acid in protein_sequence:
            if amino_acid in self.aa_to_rna:
                codons = self.aa_to_rna[amino_acid]
                chosen_codon = random.choices(codons, weights=[self.codon_probs[codon] for codon in codons])[0]
                rna_sequence += chosen_codon
            else:
                rna_sequence += "???"
        return rna_sequence

if __name__ == '__main__':
    # from get_probabability_dict()
    CODON_PROBS = {
        'UUU': 0.024639576199289373, 'UCU': 0.024567205044035097, 'UAU': 0.0246117099288076, 'UGU': 0.02463334657956687, 
        'UUC': 0.024615250299505755, 'UCC': 0.024621222659314608, 'UAC': 0.02458198574244827, 'UGC': 0.024560062998510802, 
        'UUA': 0.02468890378060863, 'UCA': 0.024574577800696145, 'UAA': 0.02482313516197096, 'UGA': 0.025301643025443965, 
        'UUG': 0.02453927044178296, 'UCG': 0.024523601179362274, 'UAG': 0.024915132828801895, 'UGG': 0.0246454877286009, 
        'CUU': 0.024603226377640886, 'CCU': 0.024536161394663455, 'CAU': 0.02467676829420141, 'CGU': 0.024375842997903675, 
        'CUC': 0.02460339577072648, 'CCC': 0.024607889438492705, 'CAC': 0.02460433459939744, 'CGC': 0.02455633317277269, 
        'CUA': 0.02476345739137613, 'CCA': 0.024562596833314437, 'CAA': 0.024506336780823694, 'CGA': 0.02472483649704897, 
        'CUG': 0.024568789819586913, 'CCG': 0.02450545157509678, 'CAG': 0.024569442079695514, 'CGG': 0.024543311409410424, 
        'AUU': 0.024597485214605374, 'ACU': 0.024549810816534203, 'AAU': 0.024648362116337367, 'AGU': 0.024522271694610572, 
        'AUC': 0.02457275306982206, 'ACC': 0.024604665513241603, 'AAC': 0.024594290776625897, 'AGC': 0.024640407994611962, 
        'AUA': 0.024625285242887398, 'ACA': 0.024571903736719372, 'AAA': 0.024556646850257793, 'AGA': 0.024662308311197898, 
        'AUG': 0.024553434411638327, 'ACG': 0.02478616850531277, 'AAG': 0.024622404999197264, 'AGG': 0.024667857576012975, 
        'GUU': 0.02452034854560896, 'GCU': 0.024530193468569356, 'GAU': 0.024620833516860192, 'GGU': 0.024706835100177068, 
        'GUC': 0.024654077784465548, 'GCC': 0.024563727798424903, 'GAC': 0.024593496930711986, 'GGC': 0.024569344762136645, 
        'GUA': 0.02467745523301079, 'GCA': 0.024554331119817367, 'GAA': 0.024625689519306538, 'GGA': 0.024631534634176923, 
        'GUG': 0.02457294667233331, 'GCG': 0.024708258902485854, 'GAG': 0.02459665522756565, 'GGG': 0.024635396137169883
    }
    # from get_dict_list()
    AA_TO_RNA = {
        'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 
        'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 
        'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': 
        ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 
        'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 
        'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']
    }

    protein_to_random_codons = ProteinToRandomRNA(CODON_PROBS, AA_TO_RNA)
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUAACUCCUAUUCAAAAUAGGGCG


### Idea of solution

We use the results of the prevouis functions to help make hte rna sequence.
convert method iterates over each amino acid in the input protein sequence. For each amino acid, it randomly selects a codon based on the probabilities specified in codon_probs and aa_to_rna. The selected codon is then added to the RNA sequence, starting at an empty string

### Discussion

The output is UUAACUCCUAUUCAAAAUAGGGCG, wich look like it could be the right solution. But without much guidence it is hard to tell. 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [2287]:
import sys
from collections import defaultdict

BASE = 'ACGT'

def sliding_window(s, k):
    """This function returns a generator that can be iterated over all
    starting positions of a k-window in the sequence.
    For each starting position, the generator returns the nucleotide frequencies
    in the window as a dictionary."""
    
    i = 0
    
    while i + k <= len(s):
        ka = s[i:i+k]
        
        
        letter_freq = {}
        for nucleotide in BASE:
            count = ka.count(nucleotide)
            letter_freq[nucleotide] = count
        
        yield letter_freq
        i += 1

if __name__ == '__main__':
    for d in sliding_window('GCGCTACGAT', 4):
        print(d)
    for s in sys.argv[1:]:
        for d in sliding_window(s, 4):
            print(d)



{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 1, 'C': 1, 'G': 1, 'T': 1}
{'A': 1, 'C': 2, 'G': 0, 'T': 1}
{'A': 1, 'C': 1, 'G': 1, 'T': 1}
{'A': 2, 'C': 1, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 1, 'T': 1}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 1, 'C': 0, 'G': 0, 'T': 0}
{'A': 1, 'C': 0, 'G': 0, 'T': 0}
{'A': 1, 'C': 0, 'G': 0, 'T': 0}
{'A': 1, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, 'C': 0, 'G': 0, 'T': 0}
{'A': 0, '

### Idea of solution

My function sliding_window generates nucleotide frequencies for each k-length substring as it moves along the input sequence. It starts at the beginning of the sequence and iterates through it, considering each window of length k. Within each window, it counts the occurrences of each nucleotide and yields these frequencies as a dictionary. This approach allows for memory-efficient processing of large sequences, providing nucleotide frequencies one window at a time

### Discussion

The top output:
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 1, 'C': 1, 'G': 1, 'T': 1}
{'A': 1, 'C': 2, 'G': 0, 'T': 1}
{'A': 1, 'C': 1, 'G': 1, 'T': 1}
{'A': 2, 'C': 1, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 1, 'T': 1}

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [2327]:
def context_list(s, k):
    contexts = {}
    for i in range(len(s) - k):
        ka = s[i:i+k]
        next_symbol = s[i+k]
        if ka in contexts:
            contexts[ka] += next_symbol
        else:
            contexts[ka] = next_symbol
    return contexts

    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution
My function context_list generates a dictionary associating each k-mer in the input DNA sequence with the symbols that follow it. It starts at the beginning of the sequence and iterates through it, considering each k-mer of length k. For each k-mer encountered, it records the symbol that immediately follows it in the sequence. If the k-mer is already present in the dictionary, the following symbol is appended to the existing list of symbols associated with that k-mer. If the k-mer is not already in the dictionary, a new entry is created with the k-mer as the key and the following symbol as the associated value.


### Discussion
when k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    the output is
{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}¨

The k-mer 'AT' appears twice in the sequence at positions 0 and 3, with following symbols 'GACCC'.
Similarly, for other k-mers like 'TG', 'GA', 'TA', 'TC', 'CA', 'CG', 'AC', and 'CT', their following symbols are recorded accordingly.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [2364]:
def context_probabilities(s, k):
    """
    Counts the frequencies of symbols in each context and converts these frequencies into probabilities.

    Args:
    - s (str): Input DNA sequence
    - k (int): Length of the k-mer
    """
    # Generate the context list
    contexts = context_list(s, k)
    
    # Calculate symbol frequencies in each context
    context_prob = {}
    for context, following_symbols in contexts.items():
        symbol_freq = {}
        for symbol in following_symbols:
            symbol_freq[symbol] = symbol_freq.get(symbol, 0) + 1
        total_symbols = sum(symbol_freq.values())
        
        # Convert frequencies to probabilities
        symbol_prob = {symbol: freq / total_symbols for symbol, freq in symbol_freq.items()}
        context_prob[context] = symbol_prob
    
    return context_prob
    
if __name__ == '__main__':
    a=context_probabilities("ATGATATCATCGACGATGTAG", 2)
    print(a)
    b = context_probabilities("ATGATATCATCGACGATGTAG", 0)
    print(b)

{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}
{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}


### Idea of solution

My function context_probabilities function aims to calculate the probabilities of observing each symbol in a context of length k within the input DNA sequence. It achieves this by first generating the contexts using the context_list function. Then, for each context, it counts the frequencies of symbols that follow that context. These frequencies are then converted into probabilities by dividing each frequency by the total number of symbols observed after the context.

### Discussion
With a=context_probabilities("ATGATATCATCGACGATGTAG", 2)
the output is
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666,
 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0},
  'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}

with context_probabilities("ATGATATCATCGACGATGTAG", 0)
{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}

My function function uses nested loops to calculate frequencies and probabilities, which may not be efficient for large sequences or contexts with many unique symbols. The function assumes that the input sequence contains valid DNA symbols and handles only contexts of length k. It does not account for cases such as empty sequences or incorrect inputs, which could lead to unexpected behavior.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [2380]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        random.seed(seed)
        sequence = ""
        current_context = "$" * self.k  # Start with context of length k
        for _ in range(n):
            if current_context in self.kth:
                next_symbol = random_event(self.kth[current_context])
            else:
                next_symbol = random_event(self.zeroth)
            sequence += next_symbol
            current_context = current_context[1:] + next_symbol
        return sequence

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

TGTATGATGA


### Idea of solution

The constructor is initialized with the zeroth-order probabilities (zeroth) and the k-th order probabilities (kth). The parameter k specifies the order of the Markov chain, with a default value of 2.

My generator method generates a random DNA sequence of length n based on the provided probabilities. It starts by setting an initial context of length k as "$" (or any other symbol not present in the original sequence). Then, for each position in the generated sequence, it randomly selects the next symbol based on the current context and the associated probabilities. The context is updated by shifting one position to the right and appending the newly selected symbol. This process continues until the desired sequence length is reached.

### Discussion
with the given information in __main__
output:TGTATGATGA

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [2382]:
import itertools

def context_pseudo_probabilities(s, k):
    kth = {}
    n = len(s)
    all_kmers = [''.join(kmer) for kmer in itertools.product("ACGT", repeat=k)]
    
    for i in range(n - k):
        kmer = s[i:i+k]
        next_n = s[i+k]
        if kmer in kth:
            kth[kmer][next_n] = kth[kmer].get(next_n, 0) + 1
        else:
            kth[kmer] = {next_n: 1}
    
    # Add pseudo counts for missing k-mers
    for kmer in all_kmers:
        if kmer not in kth:
            kth[kmer] = {'A': 1, 'C': 1, 'G': 1, 'T': 1}
    
    # Convert counts to probabilities
    for kmer, counts in kth.items():
        total_count = sum(counts.values())
        kth[kmer] = {nucleotide: count / total_count for nucleotide, count in counts.items()}
    
    return kth
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}
AT: {'G': 0.4, 'A': 0.2, 'C': 0.4}
TG: {'A': 0.5, 'T': 0.5}
GA: {'T': 0.6666666666666666, 'C': 0.3333333333333333}
TA: {'T': 0.5, 'G': 0.5}
TC: {'A': 0.5, 'G': 0.5}
CA: {'T': 1.0}
CG: {'A': 1.0}
AC: {'G': 1.0}
GT: {'A': 1.0}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
TT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

 GTAGATGTAGTATCATGTAT


### Idea of solution

Seems to do what it is suppose to do, with itertools

### Discussion

the first printout with the given values in main:
zeroth: {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}
AT: {'G': 0.4, 'A': 0.2, 'C': 0.4}
TG: {'A': 0.5, 'T': 0.5}
GA: {'T': 0.6666666666666666, 'C': 0.3333333333333333}
TA: {'T': 0.5, 'G': 0.5}
TC: {'A': 0.5, 'G': 0.5}

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [2411]:
import numpy as np

class MarkovChain:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def probability(self, s):
        prob = 1.0
        for i in range(len(s) - self.k):
            kmer = s[i:i+self.k]
            next_n = s[i+self.k]
            prob *= self.kth.get(kmer, {}).get(next_n, self.zeroth[next_n])
        return prob

if __name__ == '__main__':
    k = 2
    # Result from previous exercise
    zeroth = {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}
    kth = {'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4},
           'TG': {'A': 0.5, 'T': 0.5},
           'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333},
           'TA': {'T': 0.5, 'G': 0.5},
           'TC': {'A': 0.5, 'G': 0.5},
           'CA': {'T': 1.0},
           'CG': {'A': 1.0},
           'AC': {'G': 1.0},
           'GT': {'A': 1.0},
           'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
           'AG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
           'CC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
           'CT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
           'GC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
           'GG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
           'TT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}}
    mc = MarkovChain(k, zeroth, kth)
    sequences = ["ATGATATCATCGACGATGTAG", "ATG", "AC", "ACGT"]
    for seq in sequences:
        print(f"Probability of sequence {seq} is {mc.probability(seq)}")




Probability of sequence ATGATATCATCGACGATGTAG is 1.1851851851851853e-05
Probability of sequence ATG is 0.4
Probability of sequence AC is 1.0
Probability of sequence ACGT is 0.2857142857142857


### Idea of solution

My class takes as input the order of the Markov chain (k), the zeroth order probabilities (zeroth), and the transition probabilities for each k-mer (kth). The class provides a method probability that calculates the probability of a given DNA sequence based on the Markov chain model. The code then initializes an instance of the MarkovChain class with provided zeroth order and transition probabilities and computes the probabilities for a list of input DNA sequences.

### Discussion

Probability of sequence ATGATATCATCGACGATGTAG is 1.1851851851851853e-05
Probability of sequence ATG is 0.4
Probability of sequence AC is 1.0
Probability of sequence ACGT is 0.2857142857142857

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [2436]:
import numpy as np

class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def log_probability(self, s):
        log_prob = 0.0
        for i in range(len(s) - self.k):
            kmer = s[i:i+self.k]
            next_n = s[i+self.k]
            prob = self.kth.get(kmer, {}).get(next_n, self.zeroth[next_n])
            log_prob += np.log2(prob)
        return log_prob

if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -16.36452797660028


### Idea of solution

my class consider the conditional probabilities of observing each nucleotide given the preceding k-mer (a substring of length k) in the sequence. For each k-mer observed in the input data, my model calculates the probability of observing each possible nucleotide as the next symbol in the sequence. These probabilities are stored in the kth dictionary.
Additionally, the model also considers the probabilities of observing the first symbol in the sequence based on the zeroth order model, which are stored in the zeroth dictionary.

### Discussion
Log probability of sequence ATGATATCATCGACGATGTAG is -16.36452797660028

The output -16.36452797660028 represents the log probability of the sequence "ATGATATCATCGACGATGTAG" according to the Markov chain model defined by the provided zeroth order and transition probabilities. This value indicates the likelihood of observing the given sequence under the model.

Since log probabilities are negative, a smaller absolute value indicates a higher probability. In this case, the log probability is relatively large in magnitude, indicating that the sequence is less likely to occur according to the model.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [2452]:
def better_context_probabilities(s, k):
    contexts = {}
    for i in range(len(s) - k):
        ka = s[i:i+k]
        next_symbol = s[i+k]
        if ka in contexts:
            if next_symbol in contexts[ka]:
                contexts[ka][next_symbol] += 1
            else:
                contexts[ka][next_symbol] = 1
        else:
            contexts[ka] = {next_symbol: 1}
    return contexts

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))


AT: {'G': 2, 'A': 1, 'C': 2}
TG: {'A': 1, 'T': 1}
GA: {'T': 2, 'C': 1}
TA: {'T': 1, 'G': 1}
TC: {'A': 1, 'G': 1}
CA: {'T': 1}
CG: {'A': 2}
AC: {'G': 1}
GT: {'A': 1}


### Idea of solution
My function calculates the frequencies of symbols following each k-mer substring in the input sequence s, storing these frequencies in a nested dictionary structure. Each k-mer serves as a key in the outer dictionary, with the corresponding value being another dictionary storing the frequencies of each symbol following that k-mer. This optimization reduces the space requirement compared to storing concatenated symbols.

### Discussion
with the provided main:
  k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

i get the output
AT: {'G': 2, 'A': 1, 'C': 2}
TG: {'A': 1, 'T': 1}
GA: {'T': 2, 'C': 1}
TA: {'T': 1, 'G': 1}
TC: {'A': 1, 'G': 1}
CA: {'T': 1}
CG: {'A': 2}
AC: {'G': 1}
GT: {'A': 1}



While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [2459]:
import numpy as np

class SimpleMarkovChain:
    def __init__(self, s, k):
        self.k = k
        self.contexts = {}
        for i in range(len(s) - k):
            context = s[i:i+k]
            next_symbol = s[i+k]
            if context not in self.contexts:
                self.contexts[context] = []
            self.contexts[context].append(next_symbol)

    def generate(self, n, seed=None):
        np.random.seed(seed)
        sequence = ""
        current_context = "$" * self.k  # Start with context of length k
        for _ in range(n):
            next_symbol = np.random.choice(self.contexts.get(current_context, list(self.contexts.keys())))
            sequence += next_symbol
            current_context = current_context[1:] + next_symbol
        return sequence

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))


TCCGTATAACACACGTGTAC


### Idea of solution

ourer class samples from the concatenation of symbols following each context. The class generates a random DNA sequence of length n based on the input sequence s and the specified order k. It uses numpy.random.choice to select the next symbol based on the context.

### Discussion
with this main:
k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

Output: TCCGTATAACACACGTGTAC


## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [2475]:
def kmer_index(s, k):
    index = {}
    for i in range(len(s) - k + 1):
        kmer = s[i:i+k]
        if kmer in index:
            index[kmer].append(i)
        else:
            index[kmer] = [i]
    return index

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i % 10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d = kmer_index(s, k)
    print(d)


Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

To create a k-mer index, we need to iterate through the DNA sequence and record the positions of each k-mer encountered.

### Discussion
output
Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 
'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [2480]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return np.nan

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    pass
    
    # Maybe check that converting back to protein results in the same sequence
    pass
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities("<rna sequence>") # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

KeyError: 'pla'

### Idea of solution

Dont know how to compute this

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [2481]:
import numpy as np

def get_stationary_distributions(transition):
    """
    The function gets a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    # Transpose the transition matrix to get the form pi^T = P^T * pi^T
    transposed_transition = np.transpose(transition)
    # Get eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(transposed_transition)
    # Find the eigenvectors corresponding to eigenvalue 1
    stationary_vectors = [eigenvector.real for eigenvalue, eigenvector in zip(eigenvalues, eigenvectors.T) if np.isclose(eigenvalue, 1.0)]
    # Normalize the stationary vectors to sum up to one
    stationary_distributions = [stationary_vector / np.sum(stationary_vector) for stationary_vector in stationary_vectors]
    return stationary_distributions

if __name__ == "__main__":
    transition = np.array([[0.3, 0, 0.7, 0],
                           [0, 0.4, 0, 0.6],
                           [0.35, 0, 0.65, 0],
                           [0, 0.2, 0, 0.8]])
    # Print the stationary distributions
    for distribution in get_stationary_distributions(transition):
        print(", ".join(f"{pv:.3f}" for pv in distribution))


0.333, 0.000, 0.667, 0.000
-0.000, 0.250, -0.000, 0.750


### Idea of solution
My code calculates stationary distributions for a Markov chain by finding eigenvectors corresponding to the eigenvalue 1.0 after transposing the transition matrix. It then normalizes these eigenvectors to sum up to 1 and returns them.

### Discussion
output:

0.333, 0.000, 0.667, 0.000
-0.000, 0.250, -0.000, 0.750

The first line suggests that in the stationary state, there's a 33.3% chance of encountering nucleotide A, a 0% chance of encountering nucleotide C, a 66.7% chance of encountering nucleotide G, and a 0% chance of encountering nucleotide T.
The second line suggests that in another stationary state, there's a 0% chance of encountering nucleotide A, a 25% chance of encountering nucleotide C, a 0% chance of encountering nucleotide G, and a 75% chance of encountering nucleotide T.

21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [2511]:
import numpy as np

def kl_divergences(initial, transition):
    divergences = []
    for prefix_length in [1, 10, 100, 1000, 10000]:
        sequence = generate_sequence(initial, transition, prefix_length)
        nucleotide_distribution = {nucleotide: (sequence.count(nucleotide) + 1) / (prefix_length + 4) for nucleotide in "ACGT"}
        divergence = sum(initial[i] * np.log(initial[i] / nucleotide_distribution.get(nucleotide, 1)) for i, nucleotide in enumerate("ACGT"))
        divergences.append((prefix_length, divergence))
    return divergences


def generate_sequence(initial, transition, length):
    """
    Generates a nucleotide sequence based on the Markov model described by
    the initial distribution and the transition matrix.
    """
    nucleotides = "ACGT"
    sequence = ""
    current_state = np.random.choice(len(initial), p=initial)
    for _ in range(length):
        sequence += nucleotides[current_state]
        current_state = np.random.choice(len(initial), p=transition[current_state])
    return sequence

if __name__ == "__main__":
    transition = np.array([[0.3, 0, 0.7, 0],
                           [0, 0.4, 0, 0.6],
                           [0.35, 0, 0.65, 0],
                           [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results:
        print("KL divergence of stationary distribution prefix "
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))



Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is nan
KL divergence of stationary distribution prefix of length    10 is nan
KL divergence of stationary distribution prefix of length   100 is nan
KL divergence of stationary distribution prefix of length  1000 is nan
KL divergence of stationary distribution prefix of length 10000 is nan


C:\Users\Bruger\AppData\Local\Temp\ipykernel_4920\452923245.py:8: RuntimeWarning: divide by zero encountered in log
  divergence = sum(initial[i] * np.log(initial[i] / nucleotide_distribution.get(nucleotide, 1)) for i, nucleotide in enumerate("ACGT"))
C:\Users\Bruger\AppData\Local\Temp\ipykernel_4920\452923245.py:8: RuntimeWarning: invalid value encountered in scalar multiply
  divergence = sum(initial[i] * np.log(initial[i] / nucleotide_distribution.get(nucleotide, 1)) for i, nucleotide in enumerate("ACGT"))


### Idea of solution

im having a hard time maiking it work. i ended up with the output:

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is nan
KL divergence of stationary distribution prefix of length    10 is nan
KL divergence of stationary distribution prefix of length   100 is nan
KL divergence of stationary distribution prefix of length  1000 is nan
KL divergence of stationary distribution prefix of length 10000 is nan
C:\Users\Bruger\AppData\Local\Temp\ipykernel_4920\452923245.py:8: RuntimeWarning: divide by zero encountered in log
  divergence = sum(initial[i] * np.log(initial[i] / nucleotide_distribution.get(nucleotide, 1)) for i, nucleotide in enumerate("ACGT"))
C:\Users\Bruger\AppData\Local\Temp\ipykernel_4920\452923245.py:8: RuntimeWarning: invalid value encountered in scalar multiply
  divergence = sum(initial[i] * np.log(initial[i] / nucleotide_distribution.get(nucleotide, 1)) for i, nucleotide in enumerat

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [2512]:
import numpy as np

def get_stationary_distributions(transition):
    """
    Computes the stationary distribution(s) for a given transition matrix.
    """
    eig_values, eig_vectors = np.linalg.eig(transition.T)
    stationary_distributions = (eig_vectors[:, i] / sum(eig_vectors[:, i]) for i, val in enumerate(eig_values) if np.isclose(val, 1))
    return list(stationary_distributions)

def kl_divergences(initial, transition, equilibrium_distribution):
    """
    Calculates the KL divergences between empirical distributions
    generated using a Markov model seeded with an initial distribution and a transition 
    matrix, and the equilibrium distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    divergences = []
    for length in [1, 10, 100, 1000, 10000]:
        # Generate nucleotide sequence of specified length
        sequence = generate_sequence(transition, length)
        
        # Calculate nucleotide distribution of order 0
        nucleotide_distribution = calculate_nucleotide_distribution(sequence)
        
        # Compute KL divergence between initial distribution and nucleotide distribution
        divergence = kullback_leibler(initial, nucleotide_distribution)
        
        divergences.append((length, divergence))
    
    return divergences

def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])

if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))



Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using [-0.24897974  0.4434462  -0.33178761 -0.49943477] as initial distribution:
kl-divergence   empirical distribution
0.08944520347   [-0.18270926 -0.06952546  0.30742127 -0.09443774]
0.48448275735   [ 0.29614208  0.18900565  0.23283554 -0.27348973]
0.68320231885   [-0.01776351 -0.29758993 -0.16195288 -0.08550168]
0.71940014133   [0.47641699 0.48392635 0.43994938 0.20738138]
0.80889007385   [-0.07693297 -0.44425354  0.49478242 -0.47833372]

Using [-0.29986982 -0.04954342 -0.22555812  0.23210501] as initial distribution:
kl-divergence   empirical distribution
0.44122321698   [-0.31935872  0.02214166  0.49356486  0.45057576]
0.94313264440   [-0.0531664  -0.0044749  -0.03295864  0.001417  ]
0.72923700732   [-0.40931065 -0.36501466 -0.19973364  0.08606791]
0.91405172968   [ 0.15690412  0.2965920

### Idea of solution

We defined a function kullback_leibler to calculate the Kullback-Leibler divergence between two probability distributions represented as numpy arrays. The function ensures numerical stability by replacing zeros in the denominator with a small value and zeros in the numerator with actual zeros.

### Discussion
Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using [-0.03085001 -0.11293428 -0.40774032 -0.1747054 ] as initial distribution:
kl-divergence   empirical distribution
0.87912567037   [ 0.05711858  0.46874695 -0.18004361  0.22505456]
0.84813713256   [ 0.17303062 -0.00388629 -0.29452423  0.28659449]
0.94901083463   [ 0.2409568   0.32247516  0.16359436 -0.32011325]
0.80459110356   [-0.42862696  0.27744574 -0.18180365  0.11701867]
0.69847028917   [-0.01875517 -0.08296547  0.33704544  0.02319003]

Using [0.2130337  0.1574054  0.19329758 0.4476005 ] as initial distribution:
kl-divergence   empirical distribution
0.49150917593   [ 0.32351118  0.05664103 -0.17376028  0.36707506]
0.09127714021   [-0.49023154 -0.04286844 -0.44473069 -0.13541613]
0.49861302110   [-0.39957081  0.02590334  0.30367508 -0.33042803]
0.78043862246   [-0.17382705 -0.455183    0.40727683 -0.0527162 ]
0.94607848530   [ 0.36928607  0.47849537 -0.27941439  0.08918919]


The "kl-divergence" column indicates the KL divergence between the initial distribution and the empirical distribution for each sequence length.
The "empirical distribution" column shows the empirical distribution obtained from the generated sequences.
Each row corresponds to a different sequence length (1, 10, 100, 1000, 10000), and each block of output corresponds to a different initial distribution.